# バイナリデータの取り扱い - 2
参考：  
* https://izadori.net/python-binaryfile/
* https://tabinou.com/archives/1511
* https://atmarkit.itmedia.co.jp/ait/articles/2105/18/news019.html
* https://www.tutimogura.com/python-binaryfile-read/
* https://qiita.com/katsuki104/items/3d0fbcb5c7da19d318bd
* [Pythonのstructモジュール公式ドキュメント](https://docs.python.org/ja/3/library/struct.html)

In [1]:
import struct
import os
import time
import datetime
import numpy as np

## 文字列を検索し、ヒットしたらそこから指定バイト読みだす

In [4]:
def find_and_extract_data(filename):
    with open(filename, 'rb') as file:
        content = file.read()
        
        # AAAの位置を検索
        print(content)
        start_index = content.find(b'AAA')

        # AAAが見つかった場合
        if start_index != -1:
            end_index = start_index + 10
            extracted_data = content[start_index:end_index]
            return extracted_data
        else:
            print("AAA not found in the file.")
            return None

# 使用例
filename = 'input/binary_and_text_3.dat'  
extracted_data = find_and_extract_data(filename)

if extracted_data:
    print(extracted_data)

b'Text00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00\x00\x05\x00\x00\x00\x06\x00\x00\x00\x07\x00\x00\x00\x08\x00\x00\x00\t\x00\x00\x00AAAA000\xb5\x90\x0fe\x00\x00\x00\x00\xa9\x8a7\x00\x00\x00\x00\x00ENDText01\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00\x00\x05\x00\x00\x00\x06\x00\x00\x00\x07\x00\x00\x00\x08\x00\x00\x00\t\x00\x00\x00\n\x00\x00\x00AAAA001\xb5\x90\x0fe\x00\x00\x00\x00W\x9b7\x00\x00\x00\x00\x00ENDText02\x02\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00\x00\x05\x00\x00\x00\x06\x00\x00\x00\x07\x00\x00\x00\x08\x00\x00\x00\t\x00\x00\x00\n\x00\x00\x00\x0b\x00\x00\x00AAAA002\xb5\x90\x0fe\x00\x00\x00\x00\xcf\xa07\x00\x00\x00\x00\x00ENDText03\x03\x00\x00\x00\x04\x00\x00\x00\x05\x00\x00\x00\x06\x00\x00\x00\x07\x00\x00\x00\x08\x00\x00\x00\t\x00\x00\x00\n\x00\x00\x00\x0b\x00\x00\x00\x0c\x00\x00\x00AAAA003\xb5\x90\x0fe\x00\x00\x00\x00\xcc\xa57\x00\x00\x00\x00\x00ENDText04\x04\x00\x00\x00\x05\x00\x00\x00\x06\x00\x00\x00\x07\x00\x00\x00\x08\x00\

In [22]:
def find_and_extract_data(filename):
    extracted_chunks = []
    
    with open(filename, 'rb') as file:
        content = file.read()

        start_index = 0
        while True:
            # AAAの位置を検索
            found_index = content.find(b'AAA', start_index)
            
            # AAAが見つからなかった場合、ループを終了
            if found_index == -1:
                break

            # AAAが見つかった場所から10バイト分のデータを取得
            end_index = found_index + 6
            extracted_chunk = content[found_index:end_index]
            extracted_chunks.append(extracted_chunk)

            # 次の検索開始位置を更新
            start_index = found_index + len(b'AAA')

    return extracted_chunks

# 使用例
filename = 'input/sample_data_1.dat'  # こちらのファイル名を実際のものに置き換えてください
extracted_data_list = find_and_extract_data(filename)

for idx, data in enumerate(extracted_data_list, 1):
    print(f"Chunk {idx}: {data}")


Chunk 1: b'AAA,12'
Chunk 2: b'AAA,\r\n'
Chunk 3: b'AAA,78'
Chunk 4: b'AAA,88'


In [ ]:
with open('input/sample_data_1.dat', 'rb') as file:
    content = file.read()

In [20]:
content.find(b'AAA', 3)

13

In [ ]:
dict_epoch = {}
filename = 'input/binary_and_text_3.dat'

# １レコードあたりのバイト数
record_length = 72

# 各項目のオフセットとデータ型を定義。
# 欲しい情報がバイナリデータのどの位置にあり、どのような型（バイト長）で定義されているのか知らないと取り出せない。
block_def = {
    'epoch_s'         : {'offset': 53, 'type': '<q'},
    'epoch_msusns'         : {'offset': 61, 'type': '<l'},
}

# 結果格納用の辞書にキー：空リストを作っておく
for k in block_def.keys(): 
    dict_epoch[k] = [ ] 


with open(filename, 'rb') as file:
    contents = file.read()
    for i in range(num_records):
        for k, v in block_def.items():
            # 欲しい箇所の取り出し
            temp = struct.unpack_from(f'{v["type"]}', contents, offset= v["offset"] + record_length * i)
            dict_epoch[k].append(str(temp[0]))
        # # バイナリ部分の取り出し
        # temp = struct.unpack_from('10i', contents, offset= target_start_offset + record_length * i)
        # bin_data.append(temp)

In [119]:
# 10レコードのデータを作成する
num_records = 10

# ファイル名を指定する
filename = 'input/binary_and_text_3.dat'

# ファイルをバイナリモードで開く
with open(filename, 'wb') as file:
    for i in range(num_records):
        # テキストデータの作成。:02dは2桁でゼロパディング
        text_data_1 = f"Text{i:02d}".encode('utf-8')  # 6バイトのテキストデータを作成

        # バイナリデータの作成。iはCの型intを表し、4Byteの整数型
        binary_data_1 = struct.pack('10i', i, i+1, i+2, i+3, i+4, i+5, i+6, i+7, i+8, i+9)  # 40バイトのバイナリデータを作成

        # テキストデータの作成。:02dは2桁でゼロパディング
        text_data_2 = f"AAAA{i:03d}".encode('utf-8')  # 7バイトのテキストデータを作成

        # エポック秒の秒以上をlong longで作成
        epoch = time.time()
        print(f'エポック秒{epoch}')
        epoch_s = str(epoch).split('.')[0]
        binary_data_2 = struct.pack('q', int(epoch_s))  # 8バイト

        # エポック秒のミリ秒以下をlongで作成
        epoch_msusns = str(epoch).split('.')[1]
        binary_data_3 = struct.pack('l', int(epoch_msusns))  # 8バイト。ドキュメントでは4バイトと書かれているが、、

        # テキストデータの作成。
        text_data_3 = "END".encode('utf-8')  # 3バイトのテキストデータを作成


        # テキストデータとバイナリデータをファイルに書き込む
        file.write(text_data_1)
        file.write(binary_data_1)
        file.write(text_data_2)
        file.write(binary_data_2)
        file.write(binary_data_3)
        file.write(text_data_3)


# ファイルサイズを確認する
file_size = os.path.getsize(filename)
print(f'File size: {file_size} bytes')  # 出力: File size: 72*10 bytes


エポック秒1695518901.3639977
エポック秒1695518901.3644247
エポック秒1695518901.3645647
エポック秒1695518901.3646924
エポック秒1695518901.3648338
エポック秒1695518901.3649383
エポック秒1695518901.366534
エポック秒1695518901.3666852
エポック秒1695518901.3668692
エポック秒1695518901.367029
File size: 720 bytes


出力結果のファイルをバイナリエディタで見ると下記のようになる。  
![Alt text](image-1.png)

In [136]:
# ファイルにちゃんとエポック秒が書き込まれているか確認。リトルエンディアンなので注意。
int('00000000650F90B5', 16)

1695518901

### 作成したファイルからエポック秒、エポック秒（ミリ秒以下）だけ取り出す
やることは同じ。欲しい情報がバイナリデータのどの位置にあり、どのような型（バイト長）で定義されているのか知っていないと、  
適切に取り出すことは出来ない。

In [189]:
dict_epoch = {}
filename = 'input/binary_and_text_3.dat'

# １レコードあたりのバイト数
record_length = 72

# 各項目のオフセットとデータ型を定義。
# 欲しい情報がバイナリデータのどの位置にあり、どのような型（バイト長）で定義されているのか知らないと取り出せない。
block_def = {
    'epoch_s'         : {'offset': 53, 'type': '<q'},
    'epoch_msusns'         : {'offset': 61, 'type': '<l'},
}

# 結果格納用の辞書にキー：空リストを作っておく
for k in block_def.keys(): 
    dict_epoch[k] = [ ] 


with open(filename, 'rb') as file:
    contents = file.read()
    for i in range(num_records):
        for k, v in block_def.items():
            # 欲しい箇所の取り出し
            temp = struct.unpack_from(f'{v["type"]}', contents, offset= v["offset"] + record_length * i)
            dict_epoch[k].append(str(temp[0]))
        # # バイナリ部分の取り出し
        # temp = struct.unpack_from('10i', contents, offset= target_start_offset + record_length * i)
        # bin_data.append(temp)

In [190]:
dict_epoch

{'epoch_s': ['1695518901',
  '1695518901',
  '1695518901',
  '1695518901',
  '1695518901',
  '1695518901',
  '1695518901',
  '1695518901',
  '1695518901',
  '1695518901'],
 'epoch_msusns': ['3639977',
  '3644247',
  '3645647',
  '3646924',
  '3648338',
  '3649383',
  '366534',
  '3666852',
  '3668692',
  '367029']}

In [194]:
ziped_epoch = list(zip(dict_epoch['epoch_s'], dict_epoch['epoch_msusns']))
result = [float(v1 + '.' + v2) for v1, v2 in ziped_epoch]

In [196]:
result

[1695518901.3639977,
 1695518901.3644247,
 1695518901.3645647,
 1695518901.3646924,
 1695518901.3648338,
 1695518901.3649383,
 1695518901.366534,
 1695518901.3666852,
 1695518901.3668692,
 1695518901.367029]

In [200]:
# タイムゾーンの生成
JST = datetime.timezone(datetime.timedelta(hours=+9), 'JST')

# 取得したエポック秒を時刻に変換
datetime.datetime.fromtimestamp(result[0], tz=JST)

datetime.datetime(2023, 9, 24, 10, 28, 21, 363998, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), 'JST'))